In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from wordcloud import WordCloud
from pprint import pprint
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

stop_words = nltk.corpus.stopwords.words('english')

#tweets = pd.read_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/tweets_final.csv')
tweets = pd.read_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/mentions_tweets.csv')
tweets['clean_text'] = np.nan

remove_rt = lambda x: re.sub("RT @\w+: "," ",x)
rt = lambda x: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x).split())
tweets['clean_text'] = tweets.text.map(remove_rt).map(rt)
tweets['clean_text'] = tweets.clean_text.str.lower()

tweets[['polarity', 'subjectivity']] = tweets['clean_text'].apply(lambda x: pd.Series(TextBlob(x).sentiment))
tweets['sentiment'] = np.nan
tweets['neg'] = np.nan
tweets['neu'] = np.nan
tweets['pos'] = np.nan
tweets['compound'] = np.nan

for index, row in tweets['clean_text'].iteritems():
 score = SentimentIntensityAnalyzer().polarity_scores(row)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 if neg > pos:
  tweets.loc[index, 'sentiment'] = 'negative'
 elif pos > neg:
  tweets.loc[index, 'sentiment'] = 'positive'
 else:
  tweets.loc[index, 'sentiment'] = 'neutral'
 tweets.loc[index, 'neg'] = neg
 tweets.loc[index, 'neu'] = neu
 tweets.loc[index, 'pos'] = pos
 tweets.loc[index, 'compound'] = comp

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#tweets.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/tweets_final_sentiment.csv')
tweets.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/mentions_tweets_sentiment.csv')

tweets_pos = tweets[tweets['sentiment']=='positive']
tweets_neg = tweets[tweets['sentiment']=='negative']
tweets_neu = tweets[tweets['sentiment']=='neutral']

print(tweets_pos)
print(tweets_neg)
print(tweets_neu)

                   user_id             user  ...    pos compound
4       879870012470706176       luxetverte  ...  0.401   0.7717
9      1386499836892094464    Nancy67458658  ...  0.201   0.1280
12              3295393856      evens_jason  ...  0.152   0.3252
15     1343713894393663490      DawnSoares1  ...  0.201   0.1280
18               537728513      phil_raybon  ...  0.187   0.3182
...                    ...              ...  ...    ...      ...
36228             65640335       DorieBooks  ...  0.095   0.2500
36239  1022328182886858752         JVRusso3  ...  0.150   0.4588
36247           2166212688      ErnieWatts2  ...  0.310   0.7964
36256            843606578  healthcareicare  ...  0.061   0.0258
36263  1096907821717827585         radler52  ...  0.135   0.4215

[13570 rows x 14 columns]
                   user_id             user  ...    pos compound
2               3250859784          hartpa8  ...  0.000  -0.4019
3       825084934113079296    aplebeianlife  ...  0.000  -0.859

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts, common_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words and word not in common_words] for doc in texts]

def topic_model(tweets_var):
    tweet_sentences = tweets_var['clean_text'].values.tolist()
    tweet_words = list(sent_to_words(tweet_sentences))
    # remove stop words
    common_words = ['covid','get','amp','one','many','us','americans','still','getting','like','via','new','no','going','got','also',\
                    'even','said','would','says','go','everyone','see','since','th','much','man','make','oh','co','saying','anyone',\
                    'tell','mean','two','according','america','sure','use','look','come','around','almost','done','today','give','everyone','ever']
    tweet_words = remove_stopwords(tweet_words, common_words)


    # Create Dictionary
    id2word = corpora.Dictionary(tweet_words)
    # Create Corpus
    texts = tweet_words
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # number of topics
    num_topics = 10
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
    doc_lda = lda_model[corpus]
    return doc_lda, lda_model

In [ ]:
tweet_sentences = tweets_neg['clean_text'].values.tolist()
tweet_words = list(sent_to_words(tweet_sentences))
tweet_words = remove_stopwords(tweet_words)
x = pd.DataFrame([tweet_words]).T

NameError: ignored

In [ ]:
words_set = set(tuple(tweet) for tweet in tweet_words)
tweet_words_uq = [ list(tweet) for tweet in words_set ]
tweets_words_uq_df = pd.DataFrame([tweet_words_uq]).T
uq_words_count = []
count_temp = []
for tweet in tweets_words_uq_df[0]:
  for word in tweet:
    if word not in uq_words_count:
      uq_words_count.append(word)
      count_temp.append(1)
    else:
      count_temp[uq_words_count.index(word)] += 1


In [ ]:
uq_words_count_df = pd.DataFrame([uq_words_count]).T
uq_words_count_df[1] = np.array(count_temp)
uq_words_count_df.rename(columns={0: 'Words', 1: 'Count'}, inplace=True)
uq_words_count_df.sort_values(by='Count',inplace=True,ascending=False)
uq_words_count_df['Words'].iloc[151:200]

1456               well
1227               sure
1087         healthcare
1212             trying
973              likely
706              states
423               rates
488                 use
1283            country
983                 low
416               delta
307                look
442                 cov
18               school
1269               come
786             illegal
255              family
755              report
806             medical
579     hospitalization
285               kills
345                back
463               child
651              around
165            symptoms
738                hoax
166              almost
524                done
396                high
478               sorry
525           hospitals
888               today
1652              fully
824              person
825             doctors
1754              texas
1248               real
134                show
104                give
1168           immunity
805                keep
642             

In [ ]:
doc_lda_pos, lda_model_pos = topic_model(tweets_pos)
doc_lda_neg, lda_model_neg = topic_model(tweets_neg)
doc_lda_neu, lda_model_neu = topic_model(tweets_neu)



In [ ]:
def strip_string(string):
  string = string.lstrip("' ")
  string = string.rstrip("\" ")
  string = string.split("*\"")
  return string

def topics_populate(name,topics_df,lda):
  for i in range(lda.num_topics):
    list_terms = list(map(strip_string,lda.print_topics()[i][1].split('+')))
    for j in range(len(list_terms)):
      if list_terms[j][1] not in topics_df.index.tolist():
        new_row = np.zeros(lda.num_topics)
        new_row[i] = list_terms[j][0]
        new_row = pd.Series(new_row)
        new_row.name = list_terms[j][1]
        topics_df = topics_df.append(new_row)
      else:
        topics_df.loc[list_terms[j][1],i] = list_terms[j][0]

  #topics_df = topics_df.drop(['vaccine','covid'])
  topics_df.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/{0}.csv'.format(name))

In [ ]:
topics_pos = pd.DataFrame()
topics_neg = pd.DataFrame()
topics_neu = pd.DataFrame()

#topics_populate('topics_pos',topics_pos,lda_model_pos)
#topics_populate('topics_neg',topics_neg,lda_model_neg)
#topics_populate('topics_neu',topics_neu,lda_model_neu)
topics_populate('mentions_topics_pos',topics_pos,lda_model_pos)
topics_populate('mentions_topics_neg',topics_neg,lda_model_neg)
topics_populate('mentions_topics_neu',topics_neu,lda_model_neu)

In [ ]:
def add_topics_to_tweets(name,topics_df,tweets_var,doc):
  topic_scores = [[topic_score[1] for topic_score in tweet] for tweet in doc]
  df_topics = pd.DataFrame(topic_scores)
  max_topics = [max(tweet, key=lambda x: x[1])[0] for tweet in doc]
  max_topics_df = pd.DataFrame(max_topics)

  tweets_var.reset_index(inplace=True)
  tweets_var.drop('index',axis=1,inplace=True)
  tweets_var['topic_id'] = max_topics_df

  topics_df = pd.read_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/{0}.csv'.format(name))
  topics_df.set_index('Unnamed: 0',inplace=True)
  topics_df = topics_df.columns.to_frame().reset_index()

  tweets = tweets_var.join(topics_df[0], how='left',on='topic_id')
  tweets.rename({0:'topic'},axis=1,inplace=True)
  tweets.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/{0}_final.csv'.format(name))

In [ ]:
#add_topics_to_tweets('topics_pos',topics_pos,tweets_pos,doc_lda_pos)
#add_topics_to_tweets('topics_neg',topics_neg,tweets_neg,doc_lda_neg)
#add_topics_to_tweets('topics_neu',topics_neu,tweets_neu,doc_lda_neu)
add_topics_to_tweets('mentions_topics_pos',topics_pos,tweets_pos,doc_lda_pos)
add_topics_to_tweets('mentions_topics_neg',topics_neg,tweets_neg,doc_lda_neg)
add_topics_to_tweets('mentions_topics_neu',topics_neu,tweets_neu,doc_lda_neu)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
